<a href="https://colab.research.google.com/github/deltorobarba/chemistry/blob/main/hydrogen_H2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Computational Analysis of Hydrogen Molecule (H₂)**

**Variational Quantum Eigensolve (VQE) for Ground State Calculation**

In [1]:
!pip install pennylane -q

import pennylane as qml
from pennylane import numpy as np

# Define the molecular system
symbols = ['H', 'H']
coordinates = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.7414])  # Atomic units

# Generate the molecular Hamiltonian directly in PennyLane
H, qubits = qml.qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=0,
    mult=1,  # Singlet state
    basis='sto-3g',
    active_electrons=2,
    active_orbitals=2,
    mapping='jordan_wigner'
)

# Define the quantum device
dev = qml.device('default.qubit', wires=qubits)

# Define the ansatz (parameterized quantum circuit)
def ansatz(params):
    for i in range(qubits):
        qml.RY(params[i], wires=i)
    qml.CNOT(wires=[0, 1])

# Define the cost function (expectation value of the Hamiltonian)
@qml.qnode(dev)
def cost_fn(params):
    ansatz(params)
    return qml.expval(H)

# Perform the VQE optimization
optimizer = qml.GradientDescentOptimizer(stepsize=0.1)
params = np.random.randn(qubits)
for i in range(100):
    params = optimizer.step(cost_fn, params)

# Extract the ground state energy
ground_state_energy = cost_fn(params)

# Compute excited state energies (modify the ansatz and cost function accordingly)
# ...

print("Ground state energy:", ground_state_energy)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 978.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.8 MB/s eta 0:00:00
Ground state energy: -0.868784928020456


**Approach**
1. Molecular System Definition: Specify the atomic symbols and coordinates (in atomic units) for H₂
2. Hamiltonian Generation with PennyLane qml.qchem.molecular_hamiltonian to directly generate the qubit Hamiltonian. Specify charge, multiplicity, basis set, active electrons and orbitals, and fermion-to-qubit mapping.
3. Quantum Device, Ansatz, and Cost Function used H generated by PennyLane directly in cost function.
4. VQE Optimization to find the ground state energy.
5. Adapt the ansatz and cost function to target excited states

**Outlook**

* active_electrons and active_orbitals parameters control the size of active space, which can impact the accuracy and computational cost. Choosen based on system and desired accuracy.
* Excited state calculation: modify ansatz and cost function appropriately to target specific excited states. Techniques like subspace search or symmetry constraints might be needed.
* Extend  to more complex molecules by changing the molecular geometry and basis set
* Use more advanced ansätze like Unitary Coupled Cluster (UCC) for more accurate results

**Restricted Hartree-Fock (RHF)**

In [2]:
!pip install pyscf -q
from pyscf import gto, scf, tddft

# Define the molecular system
mol = gto.M(
    atom = 'H 0 0 0; H 0 0 0.74',  # Hydrogen molecule
    basis = '6-31g',
    verbose = 4,
)

# Perform a ground-state calculation
myhf = scf.RHF(mol).run()

# Excited-state calculation using TDDFT
mytd = tddft.TDA(myhf).run()
mytd.analyze()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 MB 17.5 MB/s eta 0:00:00
System: uname_result(system='Linux', node='5af4d54b094f', release='6.1.85+', version='#1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024', machine='x86_64')  Threads 2
Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]
numpy 1.26.4  scipy 1.13.1  h5py 3.11.0
Date: Sun Sep 29 12:45:08 2024
PySCF version 2.7.0
PySCF path  /usr/local/lib/python3.10/dist-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 2
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 H      0.000000000000   0.000000000000   0.

**Interpretation**
* The molecule has a strong absorption band around 81.31 nm (corresponding to the first excited state). This transition involves significant electron redistribution along the Z-axis.
* The second excited state is "dark" (very low oscillator strength), which means that it won't be easily observed in absorption spectroscopy.
* The third excited state has a weaker absorption band at 28.26 nm.

* **Specify atomic coordinates (of Hydrogen molecule) and basis set ('6-31g')**. The choice of basis set significantly impacts the accuracy of the results. Larger basis sets generally lead to improved accuracy but increased computational cost.
* **Restricted Hartree-Fock (RHF) calculation is performed to obtain the ground-state wavefunction**.
* **Time-dependent density functional theory (TDDFT) is employed for excited-state calculation, using Tamm-Dancoff approximation (TDA)**.
  * TDDFT is just one of several methods for excited-state calculations. Other options in PySCF include configuration interaction singles (CIS), complete active space self-consistent field (CASSCF), and equation-of-motion coupled-cluster (EOM-CC).
  * For TDDFT, selecting an appropriate functional is crucial. Some functionals are known to perform better for specific types of excited states.
* **analyze() function provides summary of calculated excitation energies and oscillator strengths**.
* There are alternative excited-state calculation methods for more advanced features: solvent effects, spin-orbit coupling, and non-adiabatic dynamics

**Singlet Excitation Energies and Oscillator Strengths:**
* Excited State 1:
  * Energy: 15.2476 eV
  * Wavelength: 81.31 nm (corresponds to the energy difference between the ground and excited state)
  * Oscillator Strength (f): 0.7700 (indicates the intensity of the transition; a higher value means a stronger absorption)
  * The transition is primarily from orbital 1 to orbital 2 (coefficient 0.70703)
* Excited State 2:
  * Energy: 28.7709 eV
  * Wavelength: 43.09 nm
  * Oscillator Strength (f): 0.0000 (this transition is "forbidden," meaning it's very unlikely to occur upon light absorption)
  * The transition is primarily from orbital 1 to orbital 3
* Excited State 3:
  * Energy: 43.8775 eV
  * Wavelength: 28.26 nm
  * Oscillator Strength (f): 0.1083 (a weaker transition compared to the first excited state)
  * The transition is primarily from orbital 1 to orbital 4


**Transition Electric/Velocity/Magnetic Dipole Moments:**
* tables describe changes in the molecule's electric and magnetic properties upon excitation.
* Electric Dipole Moments: Important for light absorption and emission.
* Velocity Dipole Moments: Related to the rate of change of the electron density during the transition.
* Magnetic Dipole Moments: Relevant for magnetic interactions and certain types of spectroscopy.
* here the first excited state has a significant electric dipole transition moment along the Z-axis,consistent with its high oscillator strength. The other states have negligible dipole moments, indicating weak or forbidden transitions.